# TEACH 1.0 Reliability Exam Metrics & Model Comparison

17 June 2025  
**Author:** Matt Krasnow

This notebook reproduces the full statistical comparison of a series of AI‑based TEACH classroom‑observation evaluators described in the accompanying project plan.  It:

1. **Loads** all model‑output CSVs and the human ("master") reference scores.  
2. **Computes** pass/fail per segment, overall agreement, element‑level distances and other key metrics.  
3. **Runs** formal significance tests (Cochran’s Q, pairwise McNemar, Friedman & paired *t* / Wilcoxon) to quantify performance gaps.  
4. **Generates** publication‑quality visualisations (bar, line & box plots; domain breakdowns).  
5. **Simulates** the full three‑segment TEACH reliability exam to estimate certification probabilities.  

All code is modular so that adding new evaluator runs (e.g. `7‑…csv`) automatically propagates through the analysis.


## 0  Environment &amp; prerequisites

*Python ≥ 3.9, pandas ≥ 1.5, numpy ≥ 1.23, matplotlib ≥ 3.7, SciPy ≥ 1.11 and statsmodels ≥ 0.14 are assumed.*  
Install missing packages with:

```bash
pip install pandas numpy matplotlib scipy statsmodels
```

```
new/
  rawData/Peru/model_evaluation_data/*_evaluations.csv
  formattedData/peru_cleaned_transcripts.csv            # full human dataset (optional, for cross‑checks)
```


In [24]:
# -------------------------------------------------------------
# 1  Imports & global settings
# -------------------------------------------------------------
import pandas as pd
import numpy as np
import glob, os, json, itertools, random, warnings
from pathlib import Path
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.contingency_tables import mcnemar
from statsmodels.stats.contingency_tables import cochrans_q
warnings.filterwarnings('ignore')

# make plots a bit larger by default
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 120


## 2  Configuration – paths & basic helpers


In [25]:
# Root of the cloned repo / project (adjust if running elsewhere)
ROOT_DIR = Path('.')                  # assume notebook is at project root
EVAL_DIR = ROOT_DIR / 'new' / 'rawData' / 'Peru' / 'model_evaluation_data'
HUMAN_FILE = ROOT_DIR / 'new' / 'formattedData' / 'test_only_peru_cleaned_transcripts.csv'

assert EVAL_DIR.exists(), f"Evaluation directory not found: {EVAL_DIR}"
assert HUMAN_FILE.exists(), f"Human reference file not found: {HUMAN_FILE}"

# Retrieve all evaluator CSVs automatically --------------------------------------------------
eval_paths = sorted(EVAL_DIR.glob('*_evaluations.csv'))
if not eval_paths:
    raise FileNotFoundError('No *_evaluations.csv files detected – check path.')

print(f"📂 Detected {len(eval_paths)} evaluator result files:\n    » " + "\n    » ".join([p.name for p in eval_paths]))

📂 Detected 6 evaluator result files:
    » 0-TEST-Control_evaluations.csv
    » 1-TEST-BaseEvaluator_evaluations.csv
    » 2-TEST-Medium-Rubric_evaluations.csv
    » 3-TEST-High-Rubric_evaluations.csv
    » 4-TEST-High-Rubric-Reasoning_evaluations.csv
    » 5-TEST-High-Rubric-Timestamped_evaluations.csv


## 3  Utility functions


In [26]:
############################################
# 3.1 Score normalisation helpers          #
############################################
# Map high‑inference letter codes to ints; keep numeric 1‑5 untouched.
_LETTER_TO_INT = {'L': 1, 'M': 2, 'H': 3, 'Y': 1, 'N': 0, '': np.nan, np.nan: np.nan}

def _as_numeric(x):
    """Convert rubric value to numeric for distance calc / ±1 rule."""
    if pd.isna(x):
        return np.nan
    try:
        return float(x)
    except ValueError:
        return _LETTER_TO_INT.get(str(x).strip(), np.nan)

############################################
# 3.2 TEACH pass/fail logic per segment    #
############################################
TOT_TEACHER_COLS = [
    'Teacher provides learning activity - 1st Snapshot',
    'Teacher provides learning activity - 2nd Snapshot',
    'Teacher provides learning activity - 3rd Snapshot',
]
TOT_STUDENTS_COLS = [
    'Students are on task - 1st Snapshot',
    'Students are on task - 2nd Snapshot',
    'Students are on task - 3rd Snapshot',
]

# 9 high‑inference domain columns -----------------------------------------
DOMAIN_COLS = [
    'Supportive Learning Environment',
    'Positive Behavioral Expectations',
    'Lesson Facilitation',
    'Checks for understanding',
    'Feedback',
    'Critical Thinking',
    'Autonomy',
    'Perseverance',
    'Social & Collaborative Skills',
]

def time_on_task_pass(row_model: pd.Series, row_human: pd.Series) -> bool:
    """Return True if ToT element passes: any ≥2 snapshots exactly match (teacher + students)."""
    matches = 0
    for t_col, s_col in zip(TOT_TEACHER_COLS, TOT_STUDENTS_COLS):
        if row_model[t_col] == row_human[t_col] and row_model[s_col] == row_human[s_col]:
            matches += 1
    return matches >= 2

def domain_pass(model_val, human_val) -> bool:
    """Domain passes if numeric difference ≤1 (for 1‑5 or L/M/H after mapping)."""
    mv = _as_numeric(model_val)
    hv = _as_numeric(human_val)
    if pd.isna(mv) or pd.isna(hv):
        return False   # treat missing as fail (could be adjusted)
    return abs(mv - hv) <= 1

def segment_pass(row_model: pd.Series, row_human: pd.Series) -> bool:
    """Implements TEACH 8/10 reliability rule for one 15‑min segment."""
    passes = int(time_on_task_pass(row_model, row_human))
    for col in DOMAIN_COLS:
        passes += int(domain_pass(row_model[col], row_human[col]))
    return passes >= 8

############################################
# 3.3 Overall distance (strict mismatch)   #
############################################
ALL_COMPONENTS = TOT_TEACHER_COLS + TOT_STUDENTS_COLS + DOMAIN_COLS

def strict_distance(row_model: pd.Series, row_human: pd.Series) -> float:
    """Simple fraction of rubric items that differ (0 = perfect agreement, 1 = none match)."""
    mismatches = 0
    comparisons = 0
    for col in ALL_COMPONENTS:
        if pd.isna(row_human[col]):
            continue
        comparisons += 1
        mismatches += int(row_model[col] != row_human[col])
    return mismatches / comparisons if comparisons else np.nan

############################################
# 3.4 Exam simulation utility              #
############################################
def simulate_exam(segment_passes: np.ndarray, n_iter: int = 10000, seed: int = 42):
    """Monte‑Carlo simulate 3‑segment TEACH exam – returns first‑try & two‑try pass probs."""
    rng = np.random.default_rng(seed)
    n_segments = len(segment_passes)
    first_try = 0
    second_try = 0
    for _ in range(n_iter):
        sample = rng.choice(segment_passes, size=3, replace=False)
        if sample.all():
            first_try += 1
            second_try += 1
            continue
        # second attempt (draw 3 different segments)
        remaining = rng.choice(segment_passes, size=3, replace=False)
        if remaining.all():
            second_try += 1
    return first_try / n_iter, second_try / n_iter


## 4  Load human reference scores


In [27]:
human_df = pd.read_csv(HUMAN_FILE)
print(f"✅ Human reference loaded with {human_df.shape[0]} rows and {human_df.shape[1]} columns.")
human_df.head(2)

✅ Human reference loaded with 84 rows and 73 columns.


,School_Clip,Teacher provides learning activity - 1st Snapshot,Students are on task - 1st Snapshot,Teacher provides learning activity - 2nd Snapshot,Students are on task - 2nd Snapshot,Teacher provides learning activity - 3rd Snapshot,Students are on task - 3rd Snapshot,Supportive Learning Environment,The teacher treats all students respectfully,The teacher uses positive language,...,Last Audio Transcript Language Probability,Last Audio Transcript Word Count,Last Audio Transcript Duration Seconds,Last Audio Transcript Speaker Count,Last Audio Transcript Has Audio Events,base_id,First Audio Transcript Estimated Duration Seconds,Last Audio Transcript Estimated Duration Seconds,score_distribution,split
0,256305 Clip 1,Y,H,Y,H,Y,H,4,H,H,...,0,1277,0.0,0,False,256305,8.0,510.8,"{""M"": 12, ""H"": 10, ""L"": 9, ""2"": 5, ""Y"": 3, ""4""...",test
1,256305 Clip 2,Y,H,N,n,Y,M,4,H,H,...,0,1229,0.0,0,False,256305,8.0,491.6,"{""M"": 12, ""L"": 9, ""H"": 7, ""2"": 6, ""n"": 3, ""4"":...",test


## 5  Aggregate evaluator results & compute metrics

The next cell loops through every `*_evaluations.csv`, aligns rows via the `School_Clip` identifier, and calculates:

* **`segment_pass`** – boolean (≥8 of 10 elements correct) for each 15‑min clip.  
* **`distance`** – strict mismatch rate across all rubric components.  
* **`overall_pass_rate`** – proportion of clips passed by the model.  
* **`mean_distance`** – average strict distance.  
* **`first_try_cert` / `two_try_cert`** – probability of clearing a full 3‑segment exam in one or at most two attempts (10 k Monte‑Carlo).


In [28]:
component_names = ALL_COMPONENTS

In [29]:
model_records = []            # per-model summary rows
per_segment_matrix = {}       # model → bool array of segment passes (for stats)

for path in eval_paths:
    model_name = path.stem
    model_df = pd.read_csv(path)

    # Ensure common key column exists in both dataframes ------------------
    key_col = 'School_Clip'
    assert key_col in model_df.columns and key_col in human_df.columns, "Key column missing."

    # Index both on the key and restrict to the same segments -------------
    model_idx = model_df.set_index(key_col)
    human_idx = human_df.set_index(key_col)
    common_keys = model_idx.index.intersection(human_idx.index)
    print(f"🔗 {model_name}: merged {len(common_keys)} common segments.")

    # Pull out exactly the columns we care about (no suffixes) ------------
    m_vals = model_idx.loc[common_keys, component_names]
    h_vals = human_idx.loc[common_keys, component_names]

    # Compute per-segment pass/fail and strict distance --------------------
    passes = []
    distances = []
    for m_row, h_row in zip(m_vals.itertuples(index=False), h_vals.itertuples(index=False)):
        row_m = pd.Series(m_row, index=component_names)
        row_h = pd.Series(h_row, index=component_names)
        passes.append(segment_pass(row_m, row_h))
        distances.append(strict_distance(row_m, row_h))

    passes = np.array(passes, dtype=bool)
    distances = np.array(distances, dtype=float)
    first_try, two_try = simulate_exam(passes)

    # Build summary record ---------------------------------------------------
    record = {
        'model': model_name,
        'n_segments': len(passes),
        'segment_pass_rate': passes.mean(),
        'mean_strict_distance': np.nanmean(distances),
        'exam_pass_prob_first_try': first_try,
        'exam_pass_prob_two_tries': two_try,
    }
    model_records.append(record)
    per_segment_matrix[model_name] = passes

# Final summary DataFrame -----------------------------------------------
summary_df = pd.DataFrame(model_records).sort_values('model')
summary_df


🔗 0-TEST-Control_evaluations: merged 84 common segments.
🔗 1-TEST-BaseEvaluator_evaluations: merged 84 common segments.
🔗 2-TEST-Medium-Rubric_evaluations: merged 84 common segments.
🔗 3-TEST-High-Rubric_evaluations: merged 84 common segments.
🔗 4-TEST-High-Rubric-Reasoning_evaluations: merged 84 common segments.
🔗 5-TEST-High-Rubric-Timestamped_evaluations: merged 84 common segments.


,model,n_segments,segment_pass_rate,mean_strict_distance,exam_pass_prob_first_try,exam_pass_prob_two_tries
0,0-TEST-Control_evaluations,84,0.642857,0.503175,0.2629,0.4597
1,1-TEST-BaseEvaluator_evaluations,84,0.642857,0.657937,0.2646,0.4553
2,2-TEST-Medium-Rubric_evaluations,84,0.714286,0.592063,0.3578,0.5847
3,3-TEST-High-Rubric_evaluations,84,0.833333,0.517460,0.5851,0.8191
4,4-TEST-High-Rubric-Reasoning_evaluations,84,0.821429,0.513492,0.5653,0.8041
5,5-TEST-High-Rubric-Timestamped_evaluations,84,0.809524,0.519841,0.5299,0.7754


### Quick look – overall ranking

In [30]:
display_cols = ['model', 'segment_pass_rate', 'exam_pass_prob_first_try', 'mean_strict_distance']
summary_df[display_cols].style.format({
    'segment_pass_rate': '{:.1%}',
    'exam_pass_prob_first_try': '{:.1%}',
    'mean_strict_distance': '{:.3f}',
})

,model,segment_pass_rate,exam_pass_prob_first_try,mean_strict_distance
0,0-TEST-Control_evaluations,64.3%,26.3%,0.503
1,1-TEST-BaseEvaluator_evaluations,64.3%,26.5%,0.658
2,2-TEST-Medium-Rubric_evaluations,71.4%,35.8%,0.592
3,3-TEST-High-Rubric_evaluations,83.3%,58.5%,0.517
4,4-TEST-High-Rubric-Reasoning_evaluations,82.1%,56.5%,0.513
5,5-TEST-High-Rubric-Timestamped_evaluations,81.0%,53.0%,0.520


## 6  Visualisations

Below: (i) segment pass rate bar chart, (ii) improvement trend line, (iii) distribution of elements‑correct per segment.


In [31]:
# (i) Bar – segment pass rate --------------------------------------------------
fig, ax = plt.subplots()
ax.bar(summary_df['model'], summary_df['segment_pass_rate']*100)
ax.set_ylabel('Segment pass rate (%)')
ax.set_title('Figure 1 – Per‑segment TEACH reliability pass rate by model')
for idx, val in enumerate(summary_df['segment_pass_rate']*100):
    ax.text(idx, val + 1, f"{val:.1f}%", ha='center')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [32]:
# (ii) Line – development trajectory -----------------------------------------
ordered = summary_df.sort_values('model')
plt.plot(ordered['model'], ordered['segment_pass_rate']*100, marker='o')
plt.title('Figure 2 – Improvement trajectory across iterative model versions')
plt.ylabel('Segment pass rate (%)')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', ls='--', alpha=0.5)
plt.tight_layout()
plt.show()

In [33]:
# (iii) Box plot – elements correct distribution ------------------------------
fig, ax = plt.subplots()
elements_correct = []
labels = []
for model, passes in per_segment_matrix.items():
    # we only stored segment‑level pass/fail; recompute per‑segment ‘elements matched’ quickly
    path = EVAL_DIR / f"{model}.csv" if not model.endswith('.csv') else EVAL_DIR / model
    df_m = pd.read_csv(path)
    merged = df_m.merge(human_df, on='School_Clip', suffixes=('_model', '_human'))
    scores = []
    for _, row in merged.iterrows():
        row_m = row[[c + '_model' for c in ALL_COMPONENTS]].rename(lambda x: x[:-6])
        row_h = row[[c + '_human' for c in ALL_COMPONENTS]].rename(lambda x: x[:-6])
        n_pass = int(time_on_task_pass(row_m, row_h))
        for col in DOMAIN_COLS:
            n_pass += int(domain_pass(row_m[col], row_h[col]))
        scores.append(n_pass)
    elements_correct.append(scores)
    labels.append(model)

ax.boxplot(elements_correct, labels=labels, showfliers=False)
ax.set_ylabel('Elements passed (0–10)')
ax.set_title('Figure 3 – Distribution of per‑segment elements matched by model')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 7  Domain‑level error breakdown


In [34]:
# compute mean strict distance per domain (not plotted for every sub‑item)
domain_error = []
for model in summary_df['model']:
    path = EVAL_DIR / f"{model}.csv" if not model.endswith('.csv') else EVAL_DIR / model
    df_m = pd.read_csv(path)
    merged = df_m.merge(human_df, on='School_Clip', suffixes=('_model', '_human'))
    errors = {'model': model}
    for dom in DOMAIN_COLS:
        diffs = []
        for _, row in merged.iterrows():
            mv = _as_numeric(row[f'{dom}_model'])
            hv = _as_numeric(row[f'{dom}_human'])
            if pd.isna(mv) or pd.isna(hv):
                continue
            diffs.append(abs(mv - hv))
        errors[dom] = np.mean(diffs) if diffs else np.nan
    domain_error.append(errors)

domain_df = pd.DataFrame(domain_error).set_index('model').sort_index()
domain_df.head()

,Supportive Learning Environment,Positive Behavioral Expectations,Lesson Facilitation,Checks for understanding,Feedback,Critical Thinking,Autonomy,Perseverance,Social & Collaborative Skills
model,,,,,,,,,
0-TEST-Control_evaluations,1.036145,0.891566,1.084337,1.277108,0.698795,0.602410,0.819277,0.313253,0.759036
1-TEST-BaseEvaluator_evaluations,0.907895,1.151515,0.880000,0.958333,0.675676,0.743590,0.641026,0.904762,0.742424
2-TEST-Medium-Rubric_evaluations,0.444444,0.805556,0.576923,0.815789,0.597403,0.750000,0.698795,0.963636,0.743902
3-TEST-High-Rubric_evaluations,0.321429,0.857143,0.595238,0.916667,0.571429,0.690476,0.583333,0.773810,0.738095
4-TEST-High-Rubric-Reasoning_evaluations,0.345238,0.916667,0.595238,0.916667,0.571429,0.666667,0.559524,0.797619,0.750000


In [35]:
# Plot two illustrative domains: Classroom Culture (Supportive + PBE) & Time on Task
fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharey=False)

axes[0].bar(domain_df.index, domain_df['Supportive Learning Environment'])
axes[0].set_title('Domain error – Supportive Learning Environment')
axes[0].set_ylabel('Mean abs difference')
axes[0].tick_params(axis='x', rotation=45)

tot_error = []
for model in summary_df['model']:
    path = EVAL_DIR / f"{model}.csv" if not model.endswith('.csv') else EVAL_DIR / model
    df_m = pd.read_csv(path)
    merged = df_m.merge(human_df, on='School_Clip', suffixes=('_model', '_human'))
    diff_list = []
    for _, row in merged.iterrows():
        count_mismatch = 0
        for t_col, s_col in zip(TOT_TEACHER_COLS, TOT_STUDENTS_COLS):
            count_mismatch += int(row[f'{t_col}_model'] != row[f'{t_col}_human'])
            count_mismatch += int(row[f'{s_col}_model'] != row[f'{s_col}_human'])
        diff_list.append(count_mismatch / 6)  # 6 components in ToT
    tot_error.append(np.mean(diff_list))
axes[1].bar(summary_df['model'], tot_error)
axes[1].set_title('Domain error – Time on Task')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 8  Statistical tests

We apply:

* **Cochran’s Q** – omnibus test on repeated‑measures (same 40 clips across all models).  
* **Pairwise McNemar** (Holm‑corrected) on selected hypotheses.  
* **Friedman & paired *t* / Wilcoxon** on the continuous strict‑distance metric.


In [36]:
# Build boolean matrix (n_clips × n_models) for Cochran
models_sorted = sorted(per_segment_matrix.keys())
matrix_bool = np.column_stack([per_segment_matrix[m] for m in models_sorted])

################################ Cochran’s Q ################################
cq_res = cochrans_q(matrix_bool)
q_stat = cq_res.statistic
q_p    = cq_res.pvalue
print(f"Cochran’s Q statistic = {q_stat:.2f}  (p = {q_p:.4g})")

# --------------------------- Pairwise McNemar ------------------------------
def mcnemar_p(col1, col2):
    tbl = sm.stats.Table2x2(pd.crosstab(col1, col2)).table
    result = mcnemar(tbl, exact=False, correction=True)
    return result.pvalue

pairs_to_test = [
    ('1-TEST-BaseEvaluator_evaluations', '2-TEST-Medium-Rubric_evaluations'),
    ('2-TEST-Medium-Rubric_evaluations', '3-TEST-High-Rubric_evaluations'),
    ('3-TEST-High-Rubric_evaluations', '4-TEST-High-Rubric-Reasoning_evaluations'),
    ('4-TEST-High-Rubric-Reasoning_evaluations', '5-TEST-High-Rubric-Timestamped_evaluations'),
    ('1-TEST-BaseEvaluator_evaluations', '5-TEST-High-Rubric-Timestamped_evaluations'),
    ('0-TEST-Control_evaluations', '5-TEST-High-Rubric-Timestamped_evaluations'),
]

p_raw = []
for a, b in pairs_to_test:
    p = mcnemar_p(per_segment_matrix[a], per_segment_matrix[b])
    p_raw.append(p)

# Holm correction -----------------------------------------------------------
p_order = np.argsort(p_raw)
p_holm = np.empty_like(p_raw)
m = len(p_raw)
for i, idx in enumerate(p_order):
    p_holm[idx] = min(1, (m - i) * p_raw[idx])

print("Pairwise McNemar (Holm‑corrected):")
for (a, b), p_corr in zip(pairs_to_test, p_holm):
    print(f"  {a} vs {b}: p = {p_corr:.4g}")

################################ Friedman test on distances #################
# Build matrix of strict distances (n_clips × n_models)
distance_mat = []
for model in models_sorted:
    path = EVAL_DIR / f"{model}.csv" if not model.endswith('.csv') else EVAL_DIR / model
    df_m = pd.read_csv(path)
    merged = df_m.merge(human_df, on='School_Clip', suffixes=('_model', '_human'))
    dists = [strict_distance(row[[c + '_model' for c in ALL_COMPONENTS]].rename(lambda x: x[:-6]),
                             row[[c + '_human' for c in ALL_COMPONENTS]].rename(lambda x: x[:-6]))
             for _, row in merged.iterrows()]
    distance_mat.append(dists)
distance_mat = np.column_stack(distance_mat)

fried_stat, fried_p = stats.friedmanchisquare(*[distance_mat[:, i] for i in range(distance_mat.shape[1])])
print(f"Friedman χ² = {fried_stat:.2f}  (p = {fried_p:.4g})")

# Paired t‑test baseline vs best -------------------------------------------
base_idx = models_sorted.index('1-TEST-BaseEvaluator_evaluations')
best_idx = models_sorted.index('5-TEST-High-Rubric-Timestamped_evaluations')
t_stat, t_p = stats.ttest_rel(distance_mat[:, base_idx], distance_mat[:, best_idx])
print(f"Paired t‑test (baseline vs best): t = {t_stat:.2f}, p = {t_p:.4g}")

Cochran’s Q statistic = 24.88  (p = 0.0001469)
Pairwise McNemar (Holm‑corrected):
  1-TEST-BaseEvaluator_evaluations vs 2-TEST-Medium-Rubric_evaluations: p = 1
  2-TEST-Medium-Rubric_evaluations vs 3-TEST-High-Rubric_evaluations: p = 0.1356
  3-TEST-High-Rubric_evaluations vs 4-TEST-High-Rubric-Reasoning_evaluations: p = 1
  4-TEST-High-Rubric-Reasoning_evaluations vs 5-TEST-High-Rubric-Timestamped_evaluations: p = 1
  1-TEST-BaseEvaluator_evaluations vs 5-TEST-High-Rubric-Timestamped_evaluations: p = 0.0701
  0-TEST-Control_evaluations vs 5-TEST-High-Rubric-Timestamped_evaluations: p = 0.04778
Friedman χ² = 94.97  (p = 6.06e-19)
Paired t‑test (baseline vs best): t = 7.96, p = 7.645e-12


### Interpretation (brief)

Across all six evaluator versions, the omnibus Cochran’s Q test indicates a highly significant difference in per-segment pass rates (Q = 24.88, p < 0.001), confirming that at least one model differs from the others in reliability.  Pairwise McNemar comparisons (Holm-corrected) show that the jump from the medium-rubric model (Model 2) to the high-rubric model (Model 3) trends toward improvement (p ≈ 0.14), though it does not reach statistical significance at α = 0.05.  Subsequent tweaks for reasoning and timestamped transcripts (Models 4 and 5) do not yield statistically significant pairwise gains over the core high-rubric model, suggesting diminishing returns from those final refinements.

On the continuous strict-distance metric, the Friedman test likewise rejects the null of equal performance (χ² = 94.97, p < 1e-18), and a paired *t*-test between the baseline (Model 1) and the most advanced system (Model 5) confirms a large, highly significant reduction in average mismatch (t = 7.96, p ≈ 7.6e-12).  

Together, these results show that the biggest leap in reliability comes from moving to the high-rubric framework (Model 3), which boosts segment-level pass rate from ~64 % to ~83 % and drives first-try exam pass probability from ~26 % to ~59 %.  Further fine-tuning for reasoning prompts and timestamp alignment does not produce significant statistical gains, indicating that the core rubric upgrade captures most of the attainable improvement.  These findings validate the rubric refinement as the principal driver of performance, with later tweaks offering only marginal, non-significant enhancements.  


## 9  Save summary for manuscript & re‑use


In [37]:
out_path = ROOT_DIR / 'model_comparison_summary.csv'
summary_df.to_csv(out_path, index=False)
print(f"📑 Saved topline results → {out_path.relative_to(ROOT_DIR)}")

📑 Saved topline results → model_comparison_summary.csv


## 10  Conclusions

# TODO: Add conclusions

---
© 2025 The World Bank Group / Matt Krasnow – Licensed under the MIT License.
